# Analysis of the effects of Covid19 on Heritage organisations

This notebook analyses the results of the effects of Covid19 on Heritage organisations.
This was available at https://historicengland.org.uk/coronavirus/heritage-sector/survey/

In [138]:
import pandas as pd
import numpy
import matplotlib as plt
from pylab import *
from plotnine import * 
from datetime import datetime


Read in csv, note commas were removed from source excel file to turn it into a csv

In [139]:
df=pd.read_csv("Covid_19_heritage.csv")

List the variables in the dataset

In [ ]:
list(df)

Relabelling some of the terms that repeat in the column names and columns to make it easier to read

In [146]:
df.columns = df.columns.map(lambda col: col.replace('Area:', ''))

df.columns = df.columns.map(lambda col: col.replace('Geography:', ''))

df.columns = df.columns.map(lambda col: col.replace('Effects:', ''))

df.columns = df.columns.map(lambda col: col.replace('Measures expected:', ''))

df= df.replace({'Area:':''}, regex=True)
df=df.replace({'Effects:':''}, regex=True)
df=df.replace({'Geography:':''}, regex=True)
df=df.replace({'Measures expected:':''}, regex=True)

# 1. Survey respondent characteristics

Heritage areas respondents covered

In [152]:
Areas=['Archaeology',
 'Archives',
 'Community heritage',
 'Conservation Architecture/Project Management',
 'Gardens and Landscapes',
 'Historic buildings (including industrial heritage sites)',
 'Intangible (e.g. cultural traditions people’s memories dialects)',
 'Maritime and transport heritage',
 'Places of worship',
 'Planning',
 'Public parks',
 'My organisation does not work in or represent any heritage areas',
 'Other']

df[Areas].count().reset_index(name='count').sort_values(['count'], ascending=False).reset_index(drop=True) 



,index,count
0,Historic buildings (including industrial herit...,552
1,Places of worship,379
2,Conservation Architecture/Project Management,313
3,Community heritage,310
4,Archaeology,275
5,Planning,214
6,Archives,195
7,Gardens and Landscapes,186
8,Intangible (e.g. cultural traditions people’s ...,143
9,Public parks,107


Regional areas

In [148]:
Regional_areas=[
 'All English regions',
 'East Midlands',
  'West Midlands',
 'East of England',
 'London',
 'South East of England',
 'South West of England',
 'North East of England',
 'North West of England']


#Regional counts
df[Regional_areas].count().reset_index(name='count').sort_values(['count'], ascending=False).reset_index(drop=True) 

,index,count
0,All English regions,194
6,South West of England,121
3,East of England,116
5,South East of England,104
4,London,95
2,West Midlands,89
8,North West of England,79
1,East Midlands,73
7,North East of England,50


# 2. Counts the effects of Covid

In [154]:
Effects=['Loss or postponement of work',
 'Business-critical staff absence',
 'Lack of available volunteers',
 'Postponement/cancellation of planned income-generating or marketing events',
 'Reduced opening/business hours',
 'Supply chain problems',
 'Increased overheads',
 'Reduced visitor numbers',
 'Have had to furlough staff',
 'Have had to lay staff off',
 'Additional costs e.g.  investment in IT/home working',
 'Delayed payment by debtors']

df[Effects].count().reset_index(name='count').sort_values(['count'], ascending=False).reset_index(drop=True) 


,index,count
0,Loss or postponement of work,618
1,Postponement/cancellation of planned income-ge...,472
2,Reduced opening/business hours,348
3,Have had to furlough staff,311
4,Reduced visitor numbers,277
5,Additional costs e.g. investment in IT/home w...,249
6,Lack of available volunteers,216
7,Delayed payment by debtors,196
8,Supply chain problems,179
9,Business-critical staff absence,135


# 3. Counts the effects of measures

In [153]:
Measures=['Coronavirus Job Retention Scheme',
 'Deferring VAT and Self-Assessment payments',
 'Self-employment Income Support Scheme',
 'Statutory Sick Pay relief package for small and medium sized businesses (SMEs)',
 '12-month business rates holiday for all retail hospitality leisure and nursery businesses in England',
 'Small business grant funding of £10000 for all business in receipt of small business rate relief or rural rate relief',
 'Grant funding of £25000 for retail hospitality and leisure businesses with property with a rateable value between £15000 and £51000',
 'Coronavirus Business Interruption Loan Scheme offering loans of up to £5 million for SMEs through the British Business Bank',
 'Lending facility from the Bank of England to help support liquidity among larger firms helping them bridge coronavirus disruption to their cash flows through loans',
 'HMRC Time To Pay Scheme',
 'Awaiting further clarification of the Government support']

df[Measures].count().reset_index(name='count').sort_values(['count'], ascending=False).reset_index(drop=True).reset_index(drop=True)  
df[Measures].rename(index={index: "x"})

,index,count
0,Awaiting further clarification of the Governme...,427
1,Coronavirus Job Retention Scheme,313
2,Deferring VAT and Self-Assessment payments,234
3,Self-employment Income Support Scheme,121
4,HMRC Time To Pay Scheme,114
5,Small business grant funding of £10000 for all...,112
6,Statutory Sick Pay relief package for small an...,86
7,12-month business rates holiday for all retail...,67
8,Coronavirus Business Interruption Loan Scheme ...,53
9,Grant funding of £25000 for retail hospitality...,37
